# Learning to combine dataframes for the U01 exRNA Healthy Control study

### Read the miRNA expression csv file and load it as a Pandas DataFrame

In [2]:
import pandas as pd
mir_cpm = pd.read_csv('get_canonical/canon_mir_logCPM.csv', index_col=0)
mir_cpm.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X129,X130,X131,X132,X133,X134,X135,X136,X137,X138
hsa-let-7a-3p,2.474925,5.587668,-1.392591,1.234309,6.082710,-1.392591,6.951551,4.005840,4.592798,4.890635,...,-1.392591,-1.392591,3.842756,-1.392591,-1.392591,3.360152,-1.392591,5.037094,0.724079,5.260851
hsa-let-7a-5p,11.821606,14.397996,16.536931,9.369801,14.857779,13.015057,14.335388,12.100140,10.986092,13.638089,...,12.223889,14.698401,14.328116,12.775353,15.199751,12.586946,11.316285,13.998248,12.396895,13.321535
hsa-let-7b-3p,5.589110,6.637579,9.625796,2.495448,7.765164,5.777791,7.535033,5.022180,2.500450,6.530026,...,5.337535,7.863207,7.071067,5.398776,7.466178,6.633097,5.022290,4.858752,2.068250,5.260851
hsa-let-7b-5p,9.568563,12.160092,13.637116,6.739557,12.145568,10.338021,11.337725,9.406264,9.962713,10.482872,...,9.641836,12.203951,11.366390,9.797243,12.366260,9.882122,9.138708,12.072477,9.447374,11.915869
hsa-let-7c-3p,-1.392591,-1.392591,-1.392591,-1.392591,-1.392591,-1.392591,-1.392591,-1.392591,-1.392591,-1.392591,...,-1.392591,-1.392591,-1.392591,-1.392591,-1.392591,-1.392591,-1.392591,-1.392591,-1.392591,-1.392591


### Load the sample information.

In [3]:
samples = pd.read_csv('sample_sheet.csv')
# capitalize & strip whitespace for consistency
for column in ['Study', 'Gender', 'Race', 'Source']:
    samples[column] = samples[column].str.capitalize()
    samples[column] = samples[column].str.strip()
# use correct ontology terms
race_ontology = {'Asian': 'Asian',
 'Black or african american': 'African American',
 'Mixed/asian & white': 'Multiracial',
 'Mixed/asian &black': 'Multiracial',
 'Mixed/black, white, asian': 'Multiracial',
 'Native hawiian or other pacific islander': 'Native Hawaiian or Other Pacific Islander',
 'Pacific islander': 'Native Hawaiian or Other Pacific Islander',
 'White': 'White'}
for id in samples.index:
    race = samples.at[id, 'Race']
    samples.at[id, 'Race'] = race_ontology[race] if race in race_ontology else 'Multiracial'
samples = samples.loc[(samples['Study'] == 'Healthy controls') & (samples['MISEQ.QC.PASS'] == 'PASS')]  # just use healthy control study for now
samples[:5]

,MT.Unique.ID,Sample.ID,Participant.ID,Study,Age,Gender,Race,Collection.Date,Source,Library.Generation.Set,Index,MiSeq.QC.Run,MISEQ.QC.PASS
0,1,70014,70014,Healthy controls,23.0,Male,Asian,5/9/2002,Plasma,Set 1a,Index1,Run1,PASS
1,2,70016,70016,Healthy controls,39.0,Male,White,5/3/2002,Plasma,Set 1a,Index2,Run1,PASS
2,3,70028,70028,Healthy controls,33.0,Female,White,5/8/2002,Plasma,Set 1a,Index3,Run1,PASS
3,4,70029,70029,Healthy controls,27.0,Female,African American,5/14/2002,Plasma,Set 1a,Index4,Run1,PASS
4,5,70038,70038,Healthy controls,22.0,Female,White,6/6/2002,Plasma,Set 1a,Index5,Run1,PASS


### Create a multi-indexed dataframe for matched biofluid samples.

In [4]:
samples_idx = samples.set_index(['Participant.ID', 'MT.Unique.ID']).sort_values(by='Participant.ID')
samples_idx[:5]

Sample.ID             Study   Age  Gender   Race  \
Participant.ID MT.Unique.ID                                                    
70014          1                70014  Healthy controls  23.0    Male  Asian   
               107              70014  Healthy controls  23.0    Male  Asian   
70016          2                70016  Healthy controls  39.0    Male  White   
               108              70016  Healthy controls  39.0    Male  White   
70028          3                70028  Healthy controls  33.0  Female  White   

                            Collection.Date  Source Library.Generation.Set  \
Participant.ID MT.Unique.ID                                                  
70014          1                   5/9/2002  Plasma                 Set 1a   
               107                 5/9/2002   Serum                  Set 5   
70016          2                   5/3/2002  Plasma                 Set 1a   
               108                 5/3/2002   Serum                  Set 5   
70028          3                   5/8/2002  Plasma                 Set 1a   

                               Index MiSeq.QC.Run MISEQ.QC.PASS  
Participant.ID MT.Unique.ID                                      
70014          1              Index1         Run1          PASS  
               107           Index13         Run3          PASS  
70016          2              Index2         Run1          PASS  
               108           Index14         Run3          PASS  
70028          3              Index3         Run1          PASS

### Remove the leading 'X' from the expression table's column ids (introduced in R).

In [5]:
mir_cpm.columns = pd.Index([i.strip('X') for i in mir_cpm.columns])
mir_cpm.iloc[:5, :5]

,1,2,3,4,5
hsa-let-7a-3p,2.474925,5.587668,-1.392591,1.234309,6.082710
hsa-let-7a-5p,11.821606,14.397996,16.536931,9.369801,14.857779
hsa-let-7b-3p,5.589110,6.637579,9.625796,2.495448,7.765164
hsa-let-7b-5p,9.568563,12.160092,13.637116,6.739557,12.145568
hsa-let-7c-3p,-1.392591,-1.392591,-1.392591,-1.392591,-1.392591


### Create a multi-index to label the miRNA ids.

In [6]:
mirna_ids = list(mir_cpm.index)
mir_cpm.index = pd.MultiIndex(levels=[['miRNA_log2CPM'], mirna_ids], labels=[[0 for i in range(len(mirna_ids))], [i for i in range(len(mirna_ids))]])
mir_cpm.iloc[:5,:5]

1          2          3         4  \
miRNA_log2CPM hsa-let-7a-3p   2.474925   5.587668  -1.392591  1.234309   
              hsa-let-7a-5p  11.821606  14.397996  16.536931  9.369801   
              hsa-let-7b-3p   5.589110   6.637579   9.625796  2.495448   
              hsa-let-7b-5p   9.568563  12.160092  13.637116  6.739557   
              hsa-let-7c-3p  -1.392591  -1.392591  -1.392591 -1.392591   

                                     5  
miRNA_log2CPM hsa-let-7a-3p   6.082710  
              hsa-let-7a-5p  14.857779  
              hsa-let-7b-3p   7.765164  
              hsa-let-7b-5p  12.145568  
              hsa-let-7c-3p  -1.392591

### Set the sample table index as the UniqueID and transform it.

In [7]:
columns_to_keep = ['Participant.ID', 'MT.Unique.ID', 'Age', 'Gender', 'Race', 'Source']
samples_t = samples[columns_to_keep].set_index('MT.Unique.ID').T
samples_t.iloc[:, :5]

MT.Unique.ID,1,2,3,4,5
Participant.ID,70014,70016,70028,70029,70038
Age,23,39,33,27,22
Gender,Male,Male,Female,Female,Female
Race,Asian,White,White,African American,White
Source,Plasma,Plasma,Plasma,Plasma,Plasma


In [8]:
row_lables = ['Participant.ID', 'Age', 'Gender', 'Race', 'Source']
samples_t.index = pd.MultiIndex(levels=[['Demographics'], row_lables], labels=[[0 for i in range(len(row_lables))], [i for i in range(len(row_lables))]])
samples_t.iloc[:, :5]

MT.Unique.ID                      1       2       3                 4       5
Demographics Participant.ID   70014   70016   70028             70029   70038
             Age                 23      39      33                27      22
             Gender            Male    Male  Female            Female  Female
             Race             Asian   White   White  African American   White
             Source          Plasma  Plasma  Plasma            Plasma  Plasma

### Attempt to combine the sample & miRNA data into one dataframe.

In [9]:
mir_cpm.columns = samples_t.columns
master_df = pd.concat([samples_t, mir_cpm], axis=1)
master_df  # this isn't working as expected... should have ~130 columns

MT.Unique.ID                        1       2       3                 4    \
Demographics  Age                    23      39      33                27   
              Gender               Male    Male  Female            Female   
              Participant.ID      70014   70016   70028             70029   
              Race                Asian   White   White  African American   
              Source             Plasma  Plasma  Plasma            Plasma   
miRNA_log2CPM hsa-let-7a-3p         NaN     NaN     NaN               NaN   
              hsa-let-7a-5p         NaN     NaN     NaN               NaN   
              hsa-let-7b-3p         NaN     NaN     NaN               NaN   
              hsa-let-7b-5p         NaN     NaN     NaN               NaN   
              hsa-let-7c-3p         NaN     NaN     NaN               NaN   
              hsa-let-7c-5p         NaN     NaN     NaN               NaN   
              hsa-let-7d-3p         NaN     NaN     NaN               NaN   
              hsa-let-7d-5p         NaN     NaN     NaN               NaN   
              hsa-let-7e-3p         NaN     NaN     NaN               NaN   
              hsa-let-7e-5p         NaN     NaN     NaN               NaN   
              hsa-let-7f-1-3p       NaN     NaN     NaN               NaN   
              hsa-let-7f-2-3p       NaN     NaN     NaN               NaN   
              hsa-let-7f-5p         NaN     NaN     NaN               NaN   
              hsa-let-7g-3p         NaN     NaN     NaN               NaN   
              hsa-let-7g-5p         NaN     NaN     NaN               NaN   
              hsa-let-7i-3p         NaN     NaN     NaN               NaN   
              hsa-let-7i-5p         NaN     NaN     NaN               NaN   
              hsa-miR-1-3p          NaN     NaN     NaN               NaN   
              hsa-miR-100-5p        NaN     NaN     NaN               NaN   
              hsa-miR-101-3p        NaN     NaN     NaN               NaN   
              hsa-miR-101-5p        NaN     NaN     NaN               NaN   
              hsa-miR-103a-2-5p     NaN     NaN     NaN               NaN   
              hsa-miR-103a-3p       NaN     NaN     NaN               NaN   
              hsa-miR-103b          NaN     NaN     NaN               NaN   
              hsa-miR-105-5p        NaN     NaN     NaN               NaN   
...                                 ...     ...     ...               ...   
              hsa-miR-9-5p          NaN     NaN     NaN               NaN   
              hsa-miR-922           NaN     NaN     NaN               NaN   
              hsa-miR-92a-1-5p      NaN     NaN     NaN               NaN   
              hsa-miR-92a-2-5p      NaN     NaN     NaN               NaN   
              hsa-miR-92a-3p        NaN     NaN     NaN               NaN   
              hsa-miR-92b-3p        NaN     NaN     NaN               NaN   
              hsa-miR-92b-5p        NaN     NaN     NaN               NaN   
              hsa-miR-93-3p         NaN     NaN     NaN               NaN   
              hsa-miR-93-5p         NaN     NaN     NaN               NaN   
              hsa-miR-933           NaN     NaN     NaN               NaN   
              hsa-miR-935           NaN     NaN     NaN               NaN   
              hsa-miR-937-3p        NaN     NaN     NaN               NaN   
              hsa-miR-937-5p        NaN     NaN     NaN               NaN   
              hsa-miR-939-3p        NaN     NaN     NaN               NaN   
              hsa-miR-939-5p        NaN     NaN     NaN               NaN   
              hsa-miR-940           NaN     NaN     NaN               NaN   
              hsa-miR-941           NaN     NaN     NaN               NaN   
              hsa-miR-942-3p        NaN     NaN     NaN               NaN   
              hsa-miR-942-5p        NaN     NaN     NaN               NaN   
              hsa-miR-943           NaN     NaN     NaN               NaN

### Successfully combine without concat() to create a master dataframe

In [10]:
import numpy as np
samples = samples_t.T  # have to transform to add new columns (adding new rows isn't straightforward)
mir_cpm_t = mir_cpm.T
assert samples.index.all() == mir_cpm_t.index.all()
master_df = samples.copy()
for mirna in mir_cpm_t.columns.levels[1]:
    master_df['miRNA_log2CPM', mirna] = mir_cpm_t['miRNA_log2CPM', 'hsa-let-7a-3p']
master_df = master_df.T  # transform the result back to the usual layout of miRNAs as rows and samples as columns
master_df

MT.Unique.ID                         1        2        3                 4    \
Demographics  Participant.ID       70014    70016    70028             70029   
              Age                     23       39       33                27   
              Gender                Male     Male   Female            Female   
              Race                 Asian    White    White  African American   
              Source              Plasma   Plasma   Plasma            Plasma   
miRNA_log2CPM hsa-let-7a-3p      2.47492  5.58767 -1.39259           1.23431   
              hsa-let-7a-5p      2.47492  5.58767 -1.39259           1.23431   
              hsa-let-7b-3p      2.47492  5.58767 -1.39259           1.23431   
              hsa-let-7b-5p      2.47492  5.58767 -1.39259           1.23431   
              hsa-let-7c-3p      2.47492  5.58767 -1.39259           1.23431   
              hsa-let-7c-5p      2.47492  5.58767 -1.39259           1.23431   
              hsa-let-7d-3p      2.47492  5.58767 -1.39259           1.23431   
              hsa-let-7d-5p      2.47492  5.58767 -1.39259           1.23431   
              hsa-let-7e-3p      2.47492  5.58767 -1.39259           1.23431   
              hsa-let-7e-5p      2.47492  5.58767 -1.39259           1.23431   
              hsa-let-7f-1-3p    2.47492  5.58767 -1.39259           1.23431   
              hsa-let-7f-2-3p    2.47492  5.58767 -1.39259           1.23431   
              hsa-let-7f-5p      2.47492  5.58767 -1.39259           1.23431   
              hsa-let-7g-3p      2.47492  5.58767 -1.39259           1.23431   
              hsa-let-7g-5p      2.47492  5.58767 -1.39259           1.23431   
              hsa-let-7i-3p      2.47492  5.58767 -1.39259           1.23431   
              hsa-let-7i-5p      2.47492  5.58767 -1.39259           1.23431   
              hsa-miR-1-3p       2.47492  5.58767 -1.39259           1.23431   
              hsa-miR-100-5p     2.47492  5.58767 -1.39259           1.23431   
              hsa-miR-101-3p     2.47492  5.58767 -1.39259           1.23431   
              hsa-miR-101-5p     2.47492  5.58767 -1.39259           1.23431   
              hsa-miR-103a-2-5p  2.47492  5.58767 -1.39259           1.23431   
              hsa-miR-103a-3p    2.47492  5.58767 -1.39259           1.23431   
              hsa-miR-103b       2.47492  5.58767 -1.39259           1.23431   
              hsa-miR-105-5p     2.47492  5.58767 -1.39259           1.23431   
...                                  ...      ...      ...               ...   
              hsa-miR-9-5p       2.47492  5.58767 -1.39259           1.23431   
              hsa-miR-922        2.47492  5.58767 -1.39259           1.23431   
              hsa-miR-92a-1-5p   2.47492  5.58767 -1.39259           1.23431   
              hsa-miR-92a-2-5p   2.47492  5.58767 -1.39259           1.23431   
              hsa-miR-92a-3p     2.47492  5.58767 -1.39259           1.23431   
              hsa-miR-92b-3p     2.47492  5.58767 -1.39259           1.23431   
              hsa-miR-92b-5p     2.47492  5.58767 -1.39259           1.23431   
              hsa-miR-93-3p      2.47492  5.58767 -1.39259           1.23431   
              hsa-miR-93-5p      2.47492  5.58767 -1.39259           1.23431   
              hsa-miR-933        2.47492  5.58767 -1.39259           1.23431   
              hsa-miR-935        2.47492  5.58767 -1.39259           1.23431   
              hsa-miR-937-3p     2.47492  5.58767 -1.39259           1.23431   
              hsa-miR-937-5p     2.47492  5.58767 -1.39259           1.23431   
              hsa-miR-939-3p     2.47492  5.58767 -1.39259           1.23431   
              hsa-miR-939-5p     2.47492  5.58767 -1.39259           1.23431   
              hsa-miR-940        2.47492  5.58767 -1.39259           1.23431   
              hsa-miR-941        2.47492  5.58767 -1.39259           1.23431   
              hsa-miR-942-3p     2.47492  5.58767 -1.39259           1.23431   


In [11]:
master_df.to_csv('master_data_healthyCtrls.csv', index=False)

### Write subsets of data for R

In [65]:
samples.columns = ['Participant.ID', 'Age', 'Gender', 'Race', 'Source']
mir_counts = pd.read_csv("get_canonical/canon_mir_counts.csv", index_col=0)
samples.index = pd.Index(['X' + str(row) for row in samples.index])

# matched plasma & serum
serum_part_ids = set(samples.loc[samples['Source'] =='Serum']['Participant.ID'])
matched_samples = samples.loc[samples['Participant.ID'].isin(serum_part_ids)]
matched_mir_counts = mir_counts
matched_mir_counts = matched_mir_counts.drop(columns=[mt_id for mt_id in mir_counts.columns if mt_id not in matched_samples.index])
matched_samples.to_csv('matched_plasma-serum_samples.csv')
matched_mir_counts.to_csv('matched_plasma-serum_mir_counts.csv')

# plasma only
plasma_samples = samples.loc[samples['Source'] == 'Plasma']
plasma_mir_counts = mir_counts.drop(columns=set(mir_counts.columns) - set(plasma_samples.index))
plasma_samples.to_csv('plasma_samples.csv')
plasma_mir_counts.to_csv('plasma_mir_counts.csv')